In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 5
fig_height = 4
fig_format = :png
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/run/media/alf/datos/misrepositorios/docencia/analisis-practicas-julia"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| output: false
using SymPy  # Para el cálculo simbólico.
using HCubature # Para el cálculo numérico de integrales múltiples.

In [3]:
using HCubature
f(x, y) = 10
f(v) = f(v...)
hcubature(f, [0,0], [1,5])

(50.0, 7.105427357601002e-15)

In [4]:
f(x, y) = 10x
f(v) = f(v...)
hcubature(f, [0,0], [1,5])

(25.0, 3.552713678800501e-15)

In [5]:
f(x, y) = x^2+y^2<1 ? 10 : 0
f(v) = f(v...)
hcubature(f, [-1,-1], [1,1])

(31.41598951201623, 4.6813472010342655e-7)

In [6]:
f(x, y) = x^2+y^2<1 ? sqrt(1-x^2-y^2) : 0
f(v) = f(v...)
hcubature(f, [-1,-1], [1,1])

(2.0943951101375893, 3.12087277156216e-8)

In [7]:
using SymPy
@syms x y
f(x, y) = x + y
integrate(f(x,y), (y, 0, 2), (x, 0, 1))

3

In [8]:
integrate(f(x,y), (x, 0, 1), (y, 0, 2))

3

In [9]:
@syms z
f(x, y, z) = x * y * z
integrate(f(x,y,z), (z, 0, 3), (y, 0, 2), (x, 0, 1))

9/2

In [10]:
@syms z
f(x, y, z) = x * y * z
integrate(f(x,y,z), (y, 0, 2), (x, 0, 1), (z, 0, 3))

9/2

In [11]:
using SymPy
@syms x y
f(x, y) = exp(x+y)
integrate(f(x,y), (y, 0, x/2), (x, 0, 2))

              3
   2   1   2⋅ℯ 
- ℯ  + ─ + ────
       3    3  

In [12]:
integrate(f(x,y), (x, 2y, 2), (y, 0, 1))

              3
   2   1   2⋅ℯ 
- ℯ  + ─ + ────
       3    3  

In [13]:
f(x, y) = x^2*y
integrate(f(x,y), (y, x, sqrt(x)), (x, 0, 1))

1/40

In [14]:
integrate(f(x,y), (x, y^2, y), (y, 0, 1))

1/40

In [15]:
f(x, y) = cos(x^2)
integrate(f(x,y), (x, y, 1), (y, 0, 1))

        ⎛                      ⎛√2⎞       ⎞                            
        ⎜                     C⎜──⎟⋅Γ(1/4)⎟                            
        ⎜  √2⋅sin(1)⋅Γ(1/4)    ⎝√π⎠       ⎟                 ⎛√2⎞       
  √2⋅√π⋅⎜- ──────────────── + ────────────⎟⋅Γ(1/4)   √2⋅√π⋅C⎜──⎟⋅Γ(1/4)
        ⎝    8⋅√π⋅Γ(5/4)        4⋅Γ(5/4)  ⎠                 ⎝√π⎠       
- ──────────────────────────────────────────────── + ──────────────────
                      8⋅Γ(5/4)                            8⋅Γ(5/4)     

In [16]:
integrate(f(x,y), (y, 0, x), (x, 0, 1))

sin(1)
──────
  2   

In [17]:
using SymPy
@syms x y z
f(x, y) = solve(x+2y+3z-6, z)[1]
g(x) = solve(f(x,y), y)
integrate(f(x,y), (y, 0, g(x)), (x, 0, 6))

6

In [18]:
using HCubature
f(x, y) = abs(x*y)
f(v) = f(v...)
hcubature(f, [-1,-1], [1,1])

(1.0, 0.0)

In [19]:
using SymPy
@syms x y
# Función de la superficie
f(x, y) = x^2/2 + y^2/2 - 10 
# Función de la región de integración
g(x, y) = 2x^2 + y^2 - 9
# Límites de integración en y
soly = solve(g(x, y), y)
# Límites de integración en x
solx = solve(g(x, 0))
integrate(f(x, y), (y, soly[1], soly[2]), (x, solx[1], solx[2]))

-1197⋅√2⋅π 
───────────
     32    

In [20]:
using SymPy
@syms x y
f(x, y) = x^2 + y^2
g(x, y) = 2x
sol = solve(f(x,y)-g(x,y), y)
integrate(g(x,y)-f(x,y), (y, sol[1], sol[2]), (x, 0, 2))

-π 
───
 2 

In [21]:
using SymPy
@syms x y r θ
f(x, y) = 2*sqrt(x^2+y^2)
g(r, θ) = f(r*cos(θ), r*sin(θ))
solr = solve(g(r,θ)-4, r)
integrate(4 - g(r,θ)*r, (r, 0, solr[2]), (θ, 0, 2PI))

16⋅π
────
 3  

In [22]:
soly = solve(f(x,y)-4, y)
solx = solve(f(x,0)-4)
integrate(sqrt(diff(f(x,y), x)^2 + diff(f(x,y), y)^2 +1), (y, soly[1], soly[2]), (x, solx[1], solx[2]))

4⋅√5⋅π

In [23]:
using SymPy
@syms x y
d(x,y) = x^2 * y^2
g(x) = 2 - x^2
h(x) = -3 + 2x^2
sol = solve(g(x)-h(x), x)
m = integrate(d(x,y), (y, h(x), g(x)), (x, sol[1], sol[2]))

460⋅√15
───────
  729  

In [24]:
# Momento con respecto a y
my = integrate(x*d(x,y), (y, h(x), g(x)), (x, sol[1], sol[2]))
# Centro de masas en x.
my / m

0

In [25]:
# Momento con respecto a x
mx = integrate(y*d(x,y), (y, h(x), g(x)), (x, sol[1], sol[2]))
# Centro de masas en y.
mx / m

-4159 
──────
 5313 

In [26]:
#| echo: false
#| output: false
using QuizQuestions, SymPy
nothing

In [27]:
#| echo: false
using SymPy
@syms x, y::real
f(x, y) = exp(x^2)
sol = N(integrate(f(x,y), (y, -x/2, 1), (x, -2, 2)))
numericq(sol, 1e-5, hint="Introducir hasta 5 decimales")

QuizQuestions.Numericq(32.90525553101446, 1.0e-5, "", "", "Introducir hasta 5 decimales", "", nothing)

In [28]:
#| echo: false
using SymPy
@syms x, y, z::real
solz = solve(z^2-4+x, z)
soly = solve(y^2-4+x, y)
solx = solve(-4+x)
sol = N(integrate(solz[2]-solz[1], (y, soly[1], soly[2]), (x, 0, solx[1])))
numericq(sol, 1e-5, hint="Introducir hasta 5 decimales")

QuizQuestions.Numericq(32, 1.0e-5, "", "", "Introducir hasta 5 decimales", "", nothing)

In [29]:
#| echo: false
using HCubature
f(x, y) = x^2+y^2 <= 1 ? log(cos(x*y)+2) : 0
f(v) = f(v...)
#sol = hcubature(f, [-1,-1], [1, 1])
numericq(3.42958314, 1e-5, hint="Introducir hasta 5 decimales")

QuizQuestions.Numericq(3.42958314, 1.0e-5, "", "", "Introducir hasta 5 decimales", "", nothing)

In [30]:
#| echo: false
using SymPy, HCubature
@syms x y
f(x, y) = sqrt(x+y)
#sol = N(integrate(sqrt(diff(f(x,y), x)^2 + diff(f(x,y), y)^2 +1), (x, 0, 1), (y, 0, 1)))
numericq(1.2823707050179478, 1e-5, hint="Introducir hasta 5 decimales")

QuizQuestions.Numericq(1.2823707050179478, 1.0e-5, "", "", "Introducir hasta 5 decimales", "", nothing)

In [31]:
#| echo: false
# using SymPy
# @syms x y
# f(x, y) = ln(x+2y+1)
# m = integrate(f(x,y), (y, x^2, sqrt(x)), (x, 0, 1))
# my = integrate(x * f(x,y), (y, x^2, sqrt(x)), (x, 0, 1))
# sol = N(my / m)
numericq(0.5313566449, 1e-5, hint="Introducir hasta 5 decimales")

QuizQuestions.Numericq(0.5313566449, 1.0e-5, "", "", "Introducir hasta 5 decimales", "", nothing)

In [32]:
#| echo: false
# mx = integrate(y * f(x,y), (y, x^2, sqrt(x)), (x, 0, 1))
# sol = N(mx / m)
numericq(0.5354903136, 1e-5, hint="Introducir hasta 5 decimales")

QuizQuestions.Numericq(0.5354903136, 1.0e-5, "", "", "Introducir hasta 5 decimales", "", nothing)